# Imports

In [103]:
import pandas as pd
import numpy as np

from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import SGDRegressor,ElasticNet,RidgeCV, LinearRegression
from sklearn.feature_selection import RFE,RFECV
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, KFold, StratifiedShuffleSplit
from sklearn.decomposition import PCA\

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Dropout, BatchNormalization, Input, GlobalMaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping,ReduceLROnPlateau
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

import seaborn as sns
import matplotlib.pyplot as plt

from collections import Counter
import os
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
sns.set()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17165284904607196673
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 4687451711350978828
physical_device_desc: "device: XLA_CPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3127299276
locality {
  bus_id: 1
  links {
  }
}
incarnation: 15240611386881885801
physical_device_desc: "device: 0, name: GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 11426148042223561868
physical_device_desc: "device: XLA_GPU device"
]


# Machine Learning

## Feature selection

In [162]:
def get_genes_RFE(X,y,estimators,num_genes_fold,num_genes_final,folds):
    genes_related = []
    
    for estimator in estimators:
        selector = RFECV(estimator, min_features_to_select=num_genes_fold, cv=folds, step=1)
        selector = selector.fit(X, y)
        ranking = selector.ranking_
        ranking_idx = ranking.argsort()[-num_genes_fold:][::-1]
        genes_related+=list(X.columns[ranking_idx])

    dic_count = Counter(genes_related)
    dic_count = {k: v for k, v in sorted(dic_count.items(), key=lambda item: item[1])}
    top_genes = list(dic_count.keys())[-num_genes_final:]
    return top_genes



## Classification

## Ajuste de hiperparâmetros

In [6]:
def get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid, model,w,h,df,plot,annot):
    
    gs = GridSearchCV(estimator=model,
                      param_grid=param_grid,
                      cv=kfold)
    best_gs = gs.fit(X_train,y_train)
    n = len(param_grid)
    scores = best_gs.cv_results_['mean_test_score']
    scores = scores.reshape(len(scores),)
    
    if n == 1:
        label = list(param_grid.keys())[0]
        par = list(param_grid.values())[0]
        plt.plot(par,scores)
        plt.ylabel('F1-score')
        plt.xlabel(label)
        plt.show()
        
    if n == 2:
        
        label1,label2 = param_grid.keys()
        par1,par2 = param_grid.values()
        scores_df = pd.DataFrame(columns=par1,index=par2)
        for i in range(len(par1)*len(par2)):
            col = i % len(par1)
            row = i // len(par1)
            scores_df.iloc[row,col] = scores[i]

        scores_df = scores_df.astype(float)
        if df:
            print(scores_df)
        if plot:
            fig= plt.figure(figsize=(w,h))
            heatmap = sns.heatmap(scores_df,annot=annot)        
            plt.xlabel(label1)
            plt.ylabel(label2)
            plt.show()

    return best_gs.best_params_

## Classification 

In [169]:
n_splits = 4
test_size = 0.3 #Hp
num_genes_fold = 15
num_genes_final = 5
folds = 10
events = os.listdir('Dados/Eventos adversos')
metrics = pd.DataFrame()
metrics_ = []

n_folds = 4
test_size = 0.35

kfold = KFold(n_folds,shuffle=True,random_state=42)

max_depth = 20
depths = [i for i in range(1,max_depth+1)]
param_grid = {'max_depth':depths,
              'splitter':['best', 'random'],
              'criterion':['gini','entropy']
             }
# pca = PCA(n_components=num_genes_final)

for event in events:
    dados = pd.read_csv(f'Dados/Eventos adversos/{event}')
    
    X = dados.iloc[:,:-1]
    y = dados.iloc[:,-1]
    genes = list(X.columns)
#     genes = get_genes_RFE(X,y,estimators,num_genes_fold,num_genes_final,folds)
    genes_dl = get_genes_dl(X,y,estimators,test_size,num_genes_fold,num_genes_final,folds)
    
    X = X[genes_dl]
#     X = pd.DataFrame(pca.fit_transform(X))
    sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)

    tree_score = 0.0

    for train_index, test_index in sss.split(X, y):    
        X_train,X_test,y_train,y_test = X.iloc[train_index],X.iloc[test_index],y.iloc[train_index],y.iloc[test_index]
        hp = get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid,DecisionTreeClassifier(random_state=42,),10,6,False,False,False)
        criterion_opt,max_depth_opt,splitter_opt = hp.values()
        #Decision Trees
        tree = make_pipeline(StandardScaler(),
                            DecisionTreeClassifier(random_state=42,
                                                   criterion=criterion_opt,
                                                   max_depth=max_depth_opt,
                                                   splitter=splitter_opt))

        tree.fit(X_train,y_train)
        pred_tree = tree.predict(X_test)
        tree_score += f1_score(y_test,pred_tree,average='weighted',pos_label="Plano ID")

    tree_score /= float(n_splits)
    
    tree_score = round(tree_score,2)
    
    metrics_.append(tree_score)

#     print(f"F1-score for {event[:-4]}: Decision Trees: {tree_score}")

metrics['Decision_trees'] = metrics_
metrics

Index(['AREG', 'BCL2', 'CASP8', 'CD3E', 'CD274', 'CLEC7A', 'CXCL13', 'FASLG',
       'FLCN1', 'FPR1', 'GBP5', 'IL12B', 'IL23A', 'MARCO', 'MRC2', 'NCAM1',
       'NLRP1', 'NLRP3', 'NLRP6', 'NLRP7', 'NLRP10', 'NLRP12', 'NOD2', 'RAB24',
       'SEC14L1', 'SPP1', 'STAT2', 'TAGAP', 'TAP1', 'TGFBR2', 'TIMP2', 'TLR1',
       'TLR2', 'TLR5', 'TLR8', 'TLR10', 'TNFRSF1A', 'TNFRSF1B'],
      dtype='object')

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Index(['AREG', 'BCL2', 'CASP8', 'CD3E', 'CD274', 'CLEC7A', 'CXCL13', 'FASLG',
       'FLCN1', 'FPR1', 'GBP5'

,Decision_trees
0,0.70
1,0.85
2,0.68
3,0.54
4,0.61
5,0.46
6,0.59
7,0.80


# Classification for # of events

In [84]:
dados = pd.read_csv('Dados/dados soma.csv')

X_soma = dados.iloc[:,:-1]
y_soma = dados.iloc[:,-1]

estimators = [SVR(kernel='linear'),
              SGDRegressor(),
              ElasticNet(),
              RidgeCV(),
              LinearRegression()]

genes = get_genes(X_soma,y_soma,estimators,num_genes_fold,num_genes_final,folds)
X_soma = X_soma[genes]

n_splits = 4
test_size = 0.3 
num_genes_fold = 15
num_genes_final = 8
folds = 10

n_folds = 4
test_size = 0.35

kfold = KFold(n_folds,shuffle=True,random_state=42)

max_depth = 20
depths = [i for i in range(1,max_depth+1)]
param_grid = {'max_depth':depths,
              'splitter':['best', 'random'],
              'criterion':['gini','entropy']
             }

sss = StratifiedShuffleSplit(n_splits=n_splits, test_size=test_size, random_state=42)

tree_score = 0.0

for train_index, test_index in sss.split(X_soma, y_soma):    
    X_train,X_test,y_train,y_test = X_soma.iloc[train_index],X_soma.iloc[test_index],y_soma.iloc[train_index],y_soma.iloc[test_index]
    hp = get_hiperparams(X_train,X_test,y_train,y_test,kfold,param_grid,DecisionTreeClassifier(random_state=42,),10,6,False,False,False)
    criterion_opt,max_depth_opt,splitter_opt = hp.values()

    #Decision Trees
    tree = make_pipeline(StandardScaler(),
                        DecisionTreeClassifier(random_state=42,
                                               criterion=criterion_opt,
                                               max_depth=max_depth_opt,
                                               splitter=splitter_opt))

    tree.fit(X_train,y_train)
    pred_tree = tree.predict(X_test)
    tree_score += f1_score(y_test,pred_tree,average='weighted',pos_label="Plano ID")

tree_score /= float(n_splits)

tree_score = round(tree_score,2)


print(f"F1-score: Decision Trees: {tree_score*100}%")

F1-score: Decision Trees: 65.0%


# Deep Learning

## Feature Selection

In [166]:
def get_genes_dl(X,y,estimators,test_size,num_genes_fold,num_genes_final,folds):
    
    input_size = X.shape[1]
    
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = test_size)
    
    model = Sequential()
    activation_dense = 'relu'
    model.add(Dense(X.shape[1], activation=activation_dense)) 
#     model.add(Dropout(0.5))
    model.add(Dense(256, activation=activation_dense)) 
#     model.add(Dropout(0.5))
    model.add(Dense(128, activation=activation_dense))
#     model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    hist= model.fit( X_train, 
                     y_train, 
                     batch_size=32, 
                     epochs=epochs, 
                     shuffle='True', 
                     validation_data=(X_test, y_test),
                     verbose=0)
    # get the first layer weights.
    weights = model.layers[1].get_weights()[0]

    # get the feature importance.
    importance = weights.sum(axis = 1)
    imp_idx = np.argsort(importance)[-num_genes_final:]
    genes_dl = list(genes[imp_idx])
    return genes_dl

## Comparison

In [168]:
epochs = 30
test_size = 0.3 
num_genes_fold = 15
num_genes_final = 8
folds = 10

estimators = [SVR(kernel='linear'),
              SGDRegressor(),
              ElasticNet(),
              RidgeCV(),
              LinearRegression()]

events = os.listdir('Dados/Eventos adversos')
for event in events:
    dados = pd.read_csv(f'Dados/Eventos adversos/{event}')    
    X = dados.iloc[:,:-1]
    y = dados.iloc[:,-1]
    genes = X.columns
    genes_ml = get_genes(X,y,estimators,num_genes_fold,num_genes_final,folds)
    X = np.array(X)
    y = np.array(y)
    y = y.reshape((y.shape[0],1))
    genes_dl = get_genes_dl(X,y,estimators,test_size,num_genes_fold,num_genes_final,folds)
    print(f'Event: {event[:-4]}\nGenes selected by RFE:{genes_ml}\nGenes selectred by MLP:{genes_dl}\n')

Event: Arthralgia
Genes selected by RFE:['TLR8', 'NLRP7', 'NLRP12', 'FLCN1', 'IL12B', 'CXCL13', 'MARCO', 'GBP5']
Genes selectred by MLP:['NLRP10', 'NOD2', 'TIMP2', 'MARCO', 'TLR5', 'FPR1', 'SPP1', 'MRC2']

Event: Arthritis
Genes selected by RFE:['NLRP1', 'CD3E', 'MRC2', 'MARCO', 'FASLG', 'IL12B', 'CD274', 'GBP5']
Genes selectred by MLP:['TAGAP', 'GBP5', 'TNFRSF1B', 'MARCO', 'NLRP7', 'NLRP6', 'FLCN1', 'NLRP1']

Event: Chills
Genes selected by RFE:['GBP5', 'FPR1', 'MRC2', 'IL23A', 'FLCN1', 'MARCO', 'IL12B', 'CXCL13']
Genes selectred by MLP:['NCAM1', 'GBP5', 'IL23A', 'NOD2', 'TLR8', 'RAB24', 'TIMP2', 'CD3E']

Event: Fatigue
Genes selected by RFE:['FLCN1', 'GBP5', 'CLEC7A', 'CD274', 'MARCO', 'IL12B', 'CXCL13', 'MRC2']
Genes selectred by MLP:['NLRP7', 'NCAM1', 'CD3E', 'NLRP1', 'STAT2', 'CD274', 'NLRP10', 'TLR10']

Event: Fever
Genes selected by RFE:['FLCN1', 'FASLG', 'MARCO', 'GBP5', 'IL12B', 'CLEC7A', 'CD274', 'CXCL13']
Genes selectred by MLP:['TGFBR2', 'TNFRSF1A', 'NLRP6', 'CXCL13', 'NLRP